check ffmode else weird syntaxis in the entire text. Example: Acta BotÄ±fmmode\acutea\elseÃ¡\finica VenezuÄ±fmmode\acutee\elseÃ©\filica  
check duplicates and remove them  


Ä±fmmode\\(\w+)\\else.*\\fi

In [16]:
import pandas as pd
import re 
import numpy as np
import os.path

import bibtexparser
from bibtexparser.bparser import BibTexParser
from bibtexparser.customization import homogenize_latex_encoding
from bibtexparser.customization import convert_to_unicode

#define parser  method for bibtex
bib_parser = BibTexParser()
bib_parser.customization = convert_to_unicode
# bib_parser.customization = homogenize_latex_encoding

from fuzzywuzzy import fuzz

## Load database

In [49]:
# load abstracts database
df_raw = pd.read_csv("../4_Abstract_extraction_Manual/DATABASE - DATABASE_preview.csv", low_memory = True)

In [50]:
df_raw

,N_Index,Old_title,Citation,Google link,Taxonomy,Systematic,Mexico,Unnamed: 7,Unnamed: 8,First gen.,...,Parsimony,Likelihood,Bayesian,MSC,Unnamed: 18,Notas,Autocheck: Abstract integrity (7),Autocheck: Abbrev. (22),Autocheck: Short abstract (22),Carlos doublecheck (170)
0,0,Andean-centred' genera in the short-branch cla...,"@article{Pirie2006Jan,\n author = {Piri...",https://scholar.google.com/scholar?&q=%27Andea...,1,1,0,1,0,1,...,1,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Mediterranean' disjunction in Senecio sect. Se...,no existe,https://scholar.google.com/scholar?&q=%27Medit...,?,Auto,?,?,?,?,...,?,?,?,?,NaN,NaN,NaN,NaN,NaN,NaN
2,4,What are natural clades in the Caesalpinia gro...,no existe,https://scholar.google.com/scholar?&q=%27What+...,?,Auto,?,?,?,?,...,?,?,?,?,NaN,NaN,NaN,NaN,NaN,NaN
3,18,5S NTS of chili peppers (Capsicum-Solanaceae):...,no existe,https://scholar.google.com/scholar?&q=5S+NTS+o...,Auto,Auto,?,?,?,?,...,?,?,?,?,NaN,NaN,NaN,NaN,NaN,NaN
4,23,A 10-gene phylogeny of Solanum section Herpyst...,"@article{Tepe2011Aug,\n author = {Tepe,...",https://scholar.google.com/scholar?&q=A+10-gen...,0,1,0,1,0,1,...,1,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4986,12153,rbcL sequences provide phylogenetic insights a...,"@article{Haufler1995Oct,\n author = {Ha...",https://scholar.google.com/scholar?&q=rbcL+seq...,?,Auto,?,?,?,Auto,...,?,?,?,?,NaN,NaN,NaN,NaN,NaN,NaN
4987,12154,rbcL sequences: a promising tool for the molec...,"@article{Dubuisson1997Oct,\n author = {...",https://scholar.google.com/scholar?&q=rbcL+seq...,?,Auto,?,?,?,Auto,...,?,?,?,?,NaN,NaN,NaN,NaN,NaN,NaN
4988,12163,trnL-F and rpl16 Sequence Data and Dense Taxon...,"@article{Del-Pino2009Mar,\n author = {D...",https://scholar.google.com/scholar?&q=trnL-F+a...,?,Auto,?,?,?,Auto,...,?,?,?,?,NaN,NaN,NaN,NaN,NaN,NaN
4989,12164,"ural Delineation, Molecular Phylogeny and Flor...","@article{Roquet2008Mar,\n author = {Roq...",https://scholar.google.com/scholar?&q=ural+Del...,?,Auto,?,?,?,?,...,?,?,?,?,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
#copy original dataframe (only first three columns)
df = df_raw.iloc[:, 0:3].copy()

#remove rows that has no bibtex citation
df = df[df["Citation"].str.startswith("@")]

In [54]:
#create final df, in which tematic columns will be added.
#in this dataframe will be the information for every citation field

final_df = pd.DataFrame()


for row in df.itertuples():
    new_row_values = {}
# for row in df.head(5).itertuples():
    #parse bibtex
    citation = bibtexparser.loads(row.Citation, bib_parser)
    citation = citation.entries[-1]
    
    #add N_Index 
    # row_values["N_Index"] = row.N_Index

    #add columns for each citation field (added prefix c. = citation)
    for field in citation:
        # #verify weird syntax for special chars 
        # citation[field] = re.sub(r"Ä±fmmode\\(\w+)\\else.*\\fi", r"\1", citation[field])


        new_row_values[f"c.{field.lower()}"] = citation[field]

    #count names and add them to the final df
    ##split names by " and "
    try:
        names = citation["author"].split(" and ")
        authors = bibtexparser.customization.getnames(names)
        n_authors = len(authors)
    except:
        authors = "ERROR"
        n_authors = "ERROR"
    
    #replace names with a organized structure from bibtexparser
    new_row_values["c.author"] = authors
    new_row_values["s.n_authors"] = str(n_authors) #in this prefix s. is added to refer statistics

    #add old title
    new_row_values["a.oldtitle"] = row.Old_title #prefix a. is for archive
    
    #compare old and new title
    try:
        titles_similitud = fuzz.token_sort_ratio(row.Old_title.lower(), citation["title"].lower())
        new_row_values["v.simil_titles"] = titles_similitud #prefix v. is for verification
    except:
        new_row_values["v.simil_titles"] = 0

    row_to_add = pd.Series(new_row_values, name=row.N_Index)
    final_df = final_df.append(row_to_add, sort=True)

In [55]:
final_df

,a.oldtitle,c.abstract,c.address,c.author,c.booktitle,c.doi,c.entrytype,c.eprint,c.id,c.isbn,...,c.note,c.number,c.pages,c.publisher,c.title,c.url,c.volume,c.year,s.n_authors,v.simil_titles
0,Andean-centred' genera in the short-branch cla...,Aim We test biogeographical hypotheses regardi...,NaN,"[Pirie, Michael D., Chatrou, Lars W., Mols, Jo...",NaN,10.1111/j.1365-2699.2005.01388.x,article,NaN,Pirie2006Jan,NaN,...,NaN,1,31--46,"John Wiley & Sons, Ltd",`Andean-centred′ genera in the short-branch cl...,NaN,33,2006,5,100.0
23,A 10-gene phylogeny of Solanum section Herpyst...,Premise of the study: Solanum section Herpysti...,NaN,"[Tepe, Eric J., Farruggia, Frank T., Bohs, Lynn]",NaN,10.3732/ajb.1000516,article,NaN,Tepe2011Aug,NaN,...,NaN,8,1356--1365,"John Wiley & Sons, Ltd",A 10-gene phylogeny of Solanum section Herpyst...,NaN,98,2011,3,100.0
26,A 250 plastome phylogeny of the grass family (...,The systematics of grasses has advanced throug...,NaN,"[Saarela, Jeffery M., Burke, Sean V., Wysocki,...",NaN,10.7717/peerj.4299,article,NaN,Saarela2018Feb,NaN,...,NaN,NaN,e4299,PeerJ Inc.,A 250 plastome phylogeny of the grass family (...,NaN,6,2018,10,100.0
29,A 313 plastome phylogenomic analysis of Pooide...,"In this study, we analyzed 313 plastid genomes...",NaN,"[Orton, Lauren M., Barberıfmmode\acutea\elseá\...",NaN,10.1016/j.ympev.2021.107110,article,NaN,Orton2021Jun,NaN,...,NaN,NaN,107110,Academic Press,A 313 plastome phylogenomic analysis of Pooide...,NaN,159,2021,7,100.0
32,A Chloroplast Phylogeny of Agavaceae subfamily...,Abstractıfmmode---\else—\fi Agavaceae subfamil...,NaN,"[Halpin, Kate M., Fishbein, Mark]",NaN,10.1600/036364413X674850,article,NaN,Halpin2013Dec,NaN,...,NaN,4,996--1011,American Society of Plant Taxonomists,A Chloroplast Phylogeny of Agavaceae subfamily...,NaN,38,2013,2,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12153,rbcL sequences provide phylogenetic insights a...,Opinions on the phylogeny and classification o...,NaN,"[Haufler, Christopher H., Ranker, Tom A.]",NaN,NaN,article,NaN,Haufler1995Oct,NaN,...,NaN,4,361--374,American Fern Society,RbcL Sequences Provide Phylogenetic Insights a...,http://www.jstor.org/stable/1547815,85,1995,2,100.0
12154,rbcL sequences: a promising tool for the molec...,A phylogenetic study usingrbcL sequences was p...,NaN,"[Dubuisson, J.-Y.]",NaN,10.1006/mpev.1997.0414,article,NaN,Dubuisson1997Oct,NaN,...,NaN,2,128--138,Academic Press,rbcL Sequences: A Promising Tool for the Molec...,NaN,8,1997,1,89.0
12163,trnL-F and rpl16 Sequence Data and Dense Taxon...,The Amaranthaceae-Chenopodiaceae alliance has ...,NaN,"[Del-Pino, Ivonne Sıfmmode\acutea\elseá\finche...",NaN,10.1600/036364409787602401,article,NaN,Del-Pino2009Mar,NaN,...,NaN,1,57--67,American Society of Plant Taxonomists,trnL-F and rpl16 Sequence Data and Dense Taxon...,NaN,34,2009,3,100.0
12164,"ural Delineation, Molecular Phylogeny and Flor...",The circumscription and infrageneric classific...,NaN,"[Roquet, Cristina, Sıfmmode\acutea\elseá\fiez,...",NaN,10.1600/036364408783887465,article,NaN,Roquet2008Mar,NaN,...,NaN,1,203--217,American Society of Plant Taxonomists,"Natural Delineation, Molecular Phylogeny and F...",NaN,33,2008,6,91.0


In [56]:
# export database as csv
# in this database tematic extraction will be done
final_df.to_csv("database_for_tematic_extraction.csv")